<a href="https://colab.research.google.com/github/nicolazilio0/deepRiccy/blob/main/TEST_SPICE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/salaniz/pycocoevalcap.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-7_3ev6ga
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-7_3ev6ga
  Resolved https://github.com/salaniz/pycocoevalcap.git to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312249 sha256=eaa5b83939ce91dfcc4e7f75bcfd29e86bd853183b36ab620227b19530ee544b
  Stored in directory: /tmp/pip-ephem-wheel-cache-avhml4xx/wheels/57/de/9f/7aa9ae75f4e4733cc4b66b5c3a65a05de62a28e8111649fee5
Successfully built pycocoevalcap


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

! tar -zxf /content/drive/MyDrive/refcocog.tar.gz

local_path = '/content/refcocog/images/'
local_annotations = '/content/refcocog/annotations/'

Mounted at /content/drive


In [3]:
from pycocoevalcap.spice.spice import Spice

import json
import pickle
import torch
from PIL import Image, ImageDraw
from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from pkg_resources import packaging
import numpy as np


In [5]:
df_captions = pd.read_json('image_captioning.json')


In [6]:
df_captions

,sentence,image_path
0,<s><s>person carring a luggage.</s>,46942
1,<s><s>the vase with the black rim.</s>,470196
2,<s><s>a white travel trailer with silver strip...,119053
3,<s><s>a man wearing glasses and a grey shirt.</s>,546847
4,<s><s>the bus with the green stripe and yellow...,418873
...,...,...
18997,<s><s>a white vase with circular bumps all ove...,196111
18998,<s><s>fruit that the lemons 3 for $1 sign cove...,450305
18999,<s><s>a boy carring a bag on his head.</s>,581789
19000,<s><s>girl in blue to the left of a identical ...,543695


In [51]:
# dataset class definition
class Coco(Dataset):
    def __init__(self, path_json, path_pickle, train=True):
        self.path_json = path_json
        self.path_pickle = path_pickle
        self.train = train

        # load images and annotations
        with open(self.path_json) as json_data:
            data = json.load(json_data)
            self.ann_frame = pd.DataFrame(data['annotations'])
            self.ann_frame = self.ann_frame.reset_index(drop=False)

        with open(self.path_pickle, 'rb') as pickle_data:
            data = pickle.load(pickle_data)
            self.refs_frame = pd.DataFrame(data)

        # separate each sentence in dataframe
        self.refs_frame = self.refs_frame.explode('sentences')
        self.refs_frame = self.refs_frame.reset_index(drop=False)

        self.size = self.refs_frame.shape[0]

        # merge the dataframes
        self.dataset = pd.merge(
            self.refs_frame, self.ann_frame, left_on='ann_id', right_on='id')
        # drop useless columns for cleaner and smaller dataset
        self.dataset = self.dataset.drop(columns=['segmentation',  'category_id_y',  'index_x',
                                         'iscrowd', 'image_id_y', 'category_id_x', 'index_y', 'area'])

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.dataset.iloc[idx]

    def get_annotation(self, idx):
        return self.ann_frame.iloc[idx]

    def get_imgframe(self, idx):
        return self.img_frame.iloc[idx]

    def get_validation(self):
        return self.dataset[self.dataset['split'] == 'val']

    def get_test(self):
        return self.dataset[self.dataset['split'] == 'test']

    def get_train(self):
        return self.dataset[self.dataset['split'] == 'train']

In [52]:
dataset = Coco(local_annotations + 'instances.json',
               local_annotations + "refs(umd).p")

In [53]:
dataset_test=dataset.get_test()

In [54]:
dataset_test

,image_id_x,split,sentences,file_name,ann_id,sent_ids,ref_id,bbox,id
0,380440,test,"{'tokens': ['the', 'man', 'in', 'yellow', 'coa...",COCO_train2014_000000380440_491042.jpg,491042,"[8, 9]",0,"[374.31, 65.06, 136.04, 201.94]",491042
1,380440,test,"{'tokens': ['skiier', 'in', 'red', 'pants'], '...",COCO_train2014_000000380440_491042.jpg,491042,"[8, 9]",0,"[374.31, 65.06, 136.04, 201.94]",491042
2,419645,test,"{'tokens': ['there', 'is', 'red', 'colored', '...",COCO_train2014_000000419645_398406.jpg,398406,"[10, 11]",1,"[93.95, 83.29, 504.61, 290.57]",398406
3,419645,test,"{'tokens': ['a', 'shiny', 'red', 'vintage', 'p...",COCO_train2014_000000419645_398406.jpg,398406,"[10, 11]",1,"[93.95, 83.29, 504.61, 290.57]",398406
4,478885,test,"{'tokens': ['a', 'apple', 'desktop', 'computer...",COCO_train2014_000000478885_124383.jpg,124383,"[14, 15]",2,"[338.8, 82.19, 147.34, 157.37]",124383
...,...,...,...,...,...,...,...,...,...
9597,63587,test,"{'tokens': ['the', 'giraffe', 'to', 'the', 'fa...",COCO_train2014_000000063587_594869.jpg,594869,"[104200, 104201]",5020,"[348.58, 230.91, 131.42, 240.23]",594869
9598,485306,test,"{'tokens': ['a', 'woman', 'in', 'green', 'look...",COCO_train2014_000000485306_1756963.jpg,1756963,"[104347, 104348]",5021,"[0.0, 59.04, 377.45, 367.96]",1756963
9599,485306,test,"{'tokens': ['a', 'woman', 'in', 'a', 'green', ...",COCO_train2014_000000485306_1756963.jpg,1756963,"[104347, 104348]",5021,"[0.0, 59.04, 377.45, 367.96]",1756963
9600,53643,test,"{'tokens': ['a', 'person', 'in', 'a', 'hawaiia...",COCO_train2014_000000053643_534940.jpg,534940,"[104462, 104463]",5022,"[212.82, 0.0, 143.08, 132.41]",534940


In [57]:
#for i in range(5):
#print(df_captions.iloc[i]['image_path'])
id=df_captions.iloc[1]['image_path']
print(id)
sentence=dataset_test.loc[dataset_test['image_id_x'] == id]
cap=df_captions.loc[df_captions['image_path'] == 398406]
#sentence=dataset_test['sentences'].where(dataset_test['ann_id'] == id)
print(sentence)

470196
Empty DataFrame
Columns: [image_id_x, split, sentences, file_name, ann_id, sent_ids, ref_id, bbox, id]
Index: []


In [ ]:
#### SPICE EVALUATION

candidate = {"1": ["a man is speaking"]}
reference = {"1": ["a man speaks."]}

scorer = Spice()
score, _ = scorer.compute_score(reference, candidate)

print("SPICE score:", score)